# How to pass run time values to a tool

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [Chat models](/docs/concepts/#chat-models)
- [LangChain Tools](/docs/concepts/#tools)
- [How to create tools](/docs/how_to/custom_tools)
- [How to use a model to call tools](https://python.langchain.com/v0.2/docs/how_to/tool_calling)
:::

:::{.callout-info} Supported models

This how-to guide uses models with native tool calling capability.
You can find a [list of all models that support tool calling](/docs/integrations/chat/).

:::

:::{.callout-info} Using with LangGraph

If you're using LangGraph, please refer to [this how-to guide](https://langchain-ai.github.io/langgraph/how-tos/pass-run-time-values-to-tools/)
which shows how to create an agent that keeps track of a given user's favorite pets.
:::

You may need to bind values to a tool that are only known at runtime. For example, the tool logic may require using the ID of the user who made the request.

Most of the time, such values should not be controlled by the LLM. In fact, allowing the LLM to control the user ID may lead to a security risk.

Instead, the LLM should only control the parameters of the tool that are meant to be controlled by the LLM, while other parameters (such as user ID) should be fixed by the application logic.

This how-to guide shows a simple design pattern that creates the tool dynamically at run time and binds to them appropriate values.

We can bind them to chat models as follows:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  fireworksParams={`model="accounts/fireworks/models/firefunction-v1", temperature=0`}
/>
```

In [1]:
# | output: false
# | echo: false

%pip install -qU langchain langchain_openai

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Passing request time information

The idea is to create the tool dynamically at request time, and bind to it the appropriate information. For example,
this information may be the user ID as resolved from the request itself.

In [2]:
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.tools import BaseTool, tool

In [3]:
user_to_pets = {}


def generate_tools_for_user(user_id: str) -> List[BaseTool]:
    """Generate a set of tools that have a user id associated with them."""

    @tool
    def update_favorite_pets(pets: List[str]) -> None:
        """Add the list of favorite pets."""
        user_to_pets[user_id] = pets

    @tool
    def delete_favorite_pets() -> None:
        """Delete the list of favorite pets."""
        if user_id in user_to_pets:
            del user_to_pets[user_id]

    @tool
    def list_favorite_pets() -> None:
        """List favorite pets if any."""
        return user_to_pets.get(user_id, [])

    return [update_favorite_pets, delete_favorite_pets, list_favorite_pets]

Verify that the tools work correctly

In [4]:
update_pets, delete_pets, list_pets = generate_tools_for_user("eugene")
update_pets.invoke({"pets": ["cat", "dog"]})
print(user_to_pets)
print(list_pets.invoke({}))

{'eugene': ['cat', 'dog']}
['cat', 'dog']


In [5]:
from langchain_core.prompts import ChatPromptTemplate


def handle_run_time_request(user_id: str, query: str):
    """Handle run time request."""
    tools = generate_tools_for_user(user_id)
    llm_with_tools = llm.bind_tools(tools)
    prompt = ChatPromptTemplate.from_messages(
        [("system", "You are a helpful assistant.")],
    )
    chain = prompt | llm_with_tools
    return llm_with_tools.invoke(query)

This code will allow the LLM to invoke the tools, but the LLM is **unaware** of the fact that a **user ID** even exists!

In [6]:
ai_message = handle_run_time_request(
    "eugene", "my favorite animals are cats and parrots."
)
ai_message.tool_calls

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_jJvjPXsNbFO5MMgW0q84iqCN'}]

:::{.callout-important}

Chat models only output requests to invoke tools, they don't actually invoke the underlying tools.

To see how to invoke the tools, please refer to [how to use a model to call tools](https://python.langchain.com/v0.2/docs/how_to/tool_calling).
:::